## SportsMonks Predictive Analysis

Using the data gathered through the [Sports Monks API](https://www.sportmonks.com/), we can begin to analyze the match and player data and try to predict match outcomes. We can do this with the [SciKit-Learn library](http://scikit-learn.org/stable/), which provides a collection of machine learning models that can be tuned to the specific problem.

Specifically, we're going to be using the [Supervised Neural Networks Classifier](http://scikit-learn.org/stable/modules/neural_networks_supervised.html) in the SciKit-Learn library.

### Setup
First, we need to set up access to the local SportsMonks database, as well as importing [Pandas](http://pandas.pydata.org/pandas-docs/version/0.23/) and [Numpy](http://www.numpy.org/).

In [ ]:
import sys
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

sys.path.insert(0, './internal')
import databaseinfo

In [2]:
engine = create_engine('mysql+mysqlconnector://{}:{}@{}/{}'.format(
    databaseinfo.db_user(),
    databaseinfo.db_passwd(),
    databaseinfo.db_host(),
    databaseinfo.db_name()))

Then, we need to retrieve and merge the Club Season data and the Starting Lineup data for each match, for both the home and away teams. After this is done, we can use this data to train the Neural Network and determine the optimal number of hidden layers and nodes that gives us the most accurate match prediction.

In [3]:
def toSqlRename(tableName, attribute, prefix):
    return "%s.%s as %s%s" % (tableName, attribute, prefix, attribute)

In [4]:
player_attributes = [
    "minutes_played",
    "appearances",
    "goals",
    "goals_conceded",
    "assists",
    "shots_on_goal",
    "shots_total",
    "fouls_committed",
    "fouls_drawn",
    "interceptions",
    "saves",
    "clearances",
    "tackles",
    "offsides",
    "blocks",
    "pen_saved",
    "pen_missed",
    "pen_scored",
    "passes_total",
    "crosses_total"
]

def player_rename(tableName, attribute, prefix):
    return "%s.%s as %s%s" % (tableName, attribute, prefix, attribute)

def home_player_rename(attribute):
    return player_rename("ls", attribute, "home_")

def away_player_rename(attribute):
    return player_rename("ls", attribute, "away_")

In [5]:
def homeRename(attribute):
    return toSqlRename("css", attribute, "home_")

def awayRename(attribute):
    return toSqlRename("css", attribute, "away_")

attributes = [
    "win_total",
    "draw_total",
    "lost_total",
    "goals_for_total",
    "goals_against_total",
    "clean_sheet_total",
    "failed_to_score_total"
]

home_string = ", ".join(list(map(homeRename, attributes)))
home_players_string = ", ".join(list(map(home_player_rename, player_attributes)))
away_string = ", ".join(list(map(awayRename, attributes)))
away_players_string = ", ".join(list(map(away_player_rename, player_attributes)))

In [6]:
test = "SELECT * \
        FROM Fixture f, ClubSeasonStats css, LineupStats ls \
        WHERE f.season_id=css.season_id \
            AND f.home_team_id=css.club_id \
            AND f.id=ls.fixture_id \
            AND f.home_team_id=ls.club_id"

club_attribute_query = "SELECT home.*, %s, %s \
FROM (  SELECT f.*, %s, %s \
        FROM Fixture f, ClubSeasonStats css, LineupStats ls \
        WHERE f.season_id=css.season_id \
            AND f.home_team_id=css.club_id \
            AND f.id=ls.fixture_id \
            AND f.home_team_id=ls.club_id \
     ) home, \
    ClubSeasonStats css, \
    LineupStats ls \
WHERE home.season_id=css.season_id \
    AND home.away_team_id=css.club_id  \
    AND home.id=ls.fixture_id \
    AND home.away_team_id=ls.club_id" % (away_string, away_players_string, home_string, home_players_string)

In [7]:
resoverall = engine.execute(club_attribute_query)
df = pd.DataFrame(resoverall.fetchall())
df.columns = resoverall.keys()

### Analysis
Now that we have the data, we need to do something with it. Initially, let's just look at the overall result of a game, giving the three outcomes a corresponding label:
* Home team wins: 'H'
* Teams draw: 'D'
* Away team wins: 'A'

In [8]:
def getResult(scores):
    home_score = scores[0]
    away_score = scores[1]
    
    if home_score > away_score:
        return 'H'
    elif home_score == away_score:
        return 'D'
    else:
        return 'A'

In [29]:
scores = df.loc[:, ['home_team_score', 'away_team_score']]
df['Result'] = scores.apply(getResult, axis=1)

After assigning these labels, we extract the input and output values and split them into the training data and the test data (80% train, 20% test). We can use Python's random library to accomplish this, combined with the array capabilities of Numpy.

In [34]:
import random

X = np.array([list(x) for x in df.loc[:, 'home_win_total':'away_crosses_total'].values])
Y = np.array(df['Result'].values)

train_range = range(0, len(X), 1)
train = random.sample(train_range, int(math.floor(len(train_range) * 0.8)))
test = [x for x in train_range if x not in train]

X_train = X[train]
Y_train = Y[train]

X_test = X[test]
Y_test = Y[test]

Now that we have the training and the test data, we can use SciKit-Learn's Pipeline and cross-validation-score imports in order to test a series of models. For these models, we can test the number of hidden layers and the number of nodes in each hidden layer.

Using the formula in Section 4.2 of a [Neural Networks paper](https://tinyurl.com/ybhoz5ea), I was able to estimate the number of optimal hidden layers and perform a smaller analysis. I also used the guidance on [this StackExchange post](https://tinyurl.com/ydhcc39y) to limit the number of hidden layers to either a single layer or two layers. Three or more hidden layers require an extremely large dataset to draw from, as well as computing power that isn't available to me.

*Note: These tests take a long time to run. The analysis I ran is summarized after this section, and the code is left here for re-use, if necessary.*

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [12]:
def get_cv_score(n1, n2):
    if n2 > 0:
        hidden_layers = (n1, n2)
    else:
        hidden_layers = (n1)
    
    scaler = StandardScaler()
    model = MLPClassifier(max_iter=500, hidden_layer_sizes=hidden_layers)
    pipeline = Pipeline([('transform', scaler), ('fit', model)])
    return cross_val_score(pipeline, X_train, Y_train, cv=10, scoring='accuracy').mean()

In [ ]:
import warnings

cv_errors = []

for n1 in range(1, 15, 1):
    for n2 in range(0, 15, 1):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            nums = ((n1, n2))
            cv_error = get_cv_error(n1, n2)
            print((nums, cv_error))
            cv_errors.append((nums, cv_error))

An overview of this analysis can be seen below. Any unmentioned combination of *(n1, n2)* had a lower cross-validation score than the below combinations:
* Single hidden layer with n1=12 = **0.5917**
* Two layers with n1=12 and n2=11 = **0.5922**
* Two layers with n1=12 and n2=14 = **0.5933**
* Two layers with n1=10 and n2=10 = **0.5943**
* Two layers with n1=11 and n2=13 = **0.5950**

Most of the other combinations landed between 0.53 and 0.58, with a gradual increase until the above values, followed by a small decrease in cross-validation score, and finally a plateau (usually around 0.578).

### Predicting the Test Dataset

Awesome, now we have the "optimal" hidden layer values for the given dataset. We can then fit the training dataset to the optimal model and predict the match results of the test dataset. Then, using some more tools from SciKit-Learn, we can view the [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) of the predictions, as well as the precision, recall, and [F1-score](https://en.wikipedia.org/wiki/F1_score) of the predictions.

In [43]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
            
    scaler = StandardScaler()
    scaler.fit(X_train)

    model = MLPClassifier(max_iter=500, hidden_layer_sizes=(11, 13))

    X_train_std = scaler.transform(X_train)
    model.fit(X_train_std, Y_train)

    X_test_std = scaler.transform(X_test)
    predictions = model.predict(X_test_std)

In [55]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(Y_test, predictions, labels=['H', 'D', 'A']))

644
[[730  77 135]
 [228 151 137]
 [216  65 363]]


In [48]:
from sklearn.metrics import classification_report

print(classification_report(Y_test,predictions))

             precision    recall  f1-score   support

          A       0.57      0.56      0.57       644
          D       0.52      0.29      0.37       516
          H       0.62      0.77      0.69       942

avg / total       0.58      0.59      0.57      2102



To interpret these:
* Correctly predicted (top-left to bottom-right diagonal):
  * 730 Home wins
  * 151 Draws
  * 363 Away wins.
  * 1244 correct out of 2102 = **59.2% correct**
* Incorrectly predicted:
  * Predicted Home wins that were wrong:
    * 228 Draws.
    * 216 Away wins.
  * Predicted Draws that were wrong:
    * 77 Home wins.
    * 65 Away wins.
  * Predicted Away wins that were wrong:
    * 135 Home wins.
    * 137 Draws.

We can see that the Neural Network heavily leans towards Home wins, which backs up the theory that [Home Field Advantage in soccer is huge](http://freakonomics.com/2011/12/18/football-freakonomics-how-advantageous-is-home-field-advantage-and-why/). We can see the results of this in the recall score: Home wins have a 77% chance of being correctly recalled, while draws only have a *29%* chance of being recalled. This huge predictive imbalance between home wins and draws is split almost directly in the middle by correctly predicted away wins.

### Curiosity About Player Statistics

After doing the above analysis, which uses both the team's statistics and the starting lineups statistics, I was curious about how much the model would be impacted if I *only* factored in the statistics for starting lineups. This analysis can be seen below, but for a quick overview:

* The cross-validation score was, on average, about 0.02 points lower than the model that uses the team's record.
* Home wins were favored even more heavily than in the above model, leading to a **0.01** recall rate for Draws.
* Classification Report average scores were about 0.05 lower than the above model.

In [56]:
X_home = df.loc[:, 'home_goals':'home_crosses_total']
X_away = df.loc[:, 'away_goals':'away_crosses_total']
X = np.array([list(x) for x in pd.merge(X_home, X_away, left_index=True, right_index=True).values])
Y = np.array(df['Result'].values)

In [57]:
train_range = range(0, len(X), 1)
train = random.sample(train_range, int(math.floor(len(train_range) * 0.8)))
test = [x for x in train_range if x not in train]

X_train = X[train]
Y_train = Y[train]

X_test = X[test]
Y_test = Y[test]

In [28]:
cv_errors = []

# Use slightly different ranges, as we now have a lower number of input values
for n1 in range(5, 12, 1):
    for n2 in range(7, 13, 1):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            nums = ((n1, n2))
            cv_error = get_cv_error(n1, n2)
            print(nums)
            print(cv_error)
            cv_errors.append((nums, cv_error))
            
cv_errors

(5, 7)
0.5665457312060033
(5, 8)
0.563098300979191
(5, 9)
0.5639330498633124
(5, 10)
0.5678551222244337
(5, 11)
0.5664286670442823
(5, 12)
0.567854983024441
(6, 7)
0.5657186215866863
(6, 8)
0.5685668521965888
(6, 9)
0.5690457389574373
(6, 10)
0.5626208395343661
(6, 11)
0.5645291389735635
(6, 12)
0.5660705363317885
(7, 7)
0.564288360585607
(7, 8)
0.5652335175135071
(7, 9)
0.5692787348421724
(7, 10)
0.5659554448534541
(7, 11)
0.562032247793037
(7, 12)
0.5671419632232455
(8, 7)
0.5649999564205272
(8, 8)
0.5652438523856909
(8, 9)
0.5634540282879504
(8, 10)
0.5660730819652352
(8, 11)
0.5629813780348712
(8, 12)
0.5663132927901016
(9, 7)
0.5640487001590261
(9, 8)
0.5665482738133372
(9, 9)
0.5646360168896899
(9, 10)
0.5629785489531086
(9, 11)
0.5670241831996642
(9, 12)
0.5629771307040266
(10, 7)
0.5621450686659439
(10, 8)
0.5638118853149593
(10, 9)
0.5670231977075748
(10, 10)
0.5665523893509724
(10, 11)
0.5678562523022833
(10, 12)
0.5622665233993314
(11, 7)
0.5651182922161386
(11, 8)
0.5638098

[((5, 7), 0.5665457312060033),
 ((5, 8), 0.563098300979191),
 ((5, 9), 0.5639330498633124),
 ((5, 10), 0.5678551222244337),
 ((5, 11), 0.5664286670442823),
 ((5, 12), 0.567854983024441),
 ((6, 7), 0.5657186215866863),
 ((6, 8), 0.5685668521965888),
 ((6, 9), 0.5690457389574373),
 ((6, 10), 0.5626208395343661),
 ((6, 11), 0.5645291389735635),
 ((6, 12), 0.5660705363317885),
 ((7, 7), 0.564288360585607),
 ((7, 8), 0.5652335175135071),
 ((7, 9), 0.5692787348421724),
 ((7, 10), 0.5659554448534541),
 ((7, 11), 0.562032247793037),
 ((7, 12), 0.5671419632232455),
 ((8, 7), 0.5649999564205272),
 ((8, 8), 0.5652438523856909),
 ((8, 9), 0.5634540282879504),
 ((8, 10), 0.5660730819652352),
 ((8, 11), 0.5629813780348712),
 ((8, 12), 0.5663132927901016),
 ((9, 7), 0.5640487001590261),
 ((9, 8), 0.5665482738133372),
 ((9, 9), 0.5646360168896899),
 ((9, 10), 0.5629785489531086),
 ((9, 11), 0.5670241831996642),
 ((9, 12), 0.5629771307040266),
 ((10, 7), 0.5621450686659439),
 ((10, 8), 0.56381188531495

In [72]:
scaler = StandardScaler()
scaler.fit(X_train)

model = MLPClassifier(max_iter=500, hidden_layer_sizes=(7, 9))

X_train_std = scaler.transform(X_train)
model.fit(X_train_std, Y_train)

X_test_std = scaler.transform(X_test)
predictions = model.predict(X_test_std)

/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [73]:
print(confusion_matrix(Y_test, predictions, labels=['H', 'D', 'A']))
print(classification_report(Y_test,predictions))

[[781   4 147]
 [341   6 193]
 [245   2 383]]
             precision    recall  f1-score   support

          A       0.53      0.61      0.57       630
          D       0.50      0.01      0.02       540
          H       0.57      0.84      0.68       932

avg / total       0.54      0.56      0.48      2102

